# Evaluate Term Dispersion Scores on the GENIA Corpus Data and Reproduce Results Reported in the Mannuscript 

Authors: Samuel Sarria Hurtado, Uyen "Rachel" Lai, and Paul Sheridan

Last update: 2025-06-03

Description: Evaluate the following term dispersion scores on the Genia corpus data:
- Inverse Document Frequency (IDF)
- Inverse Collection Frequency (ICF)
- Chi-square
- Church and Gale (CG)
- Irvine and Callison-Burch (ICB)
- Derivation of Proportions (DoP)
- Residual ICF (RICF)

Calculate average P@k scores for each scoring function using the GENIA terms as ground truth. Also, evaluate scoring functions for their ability to filter out stopwords. 

## Preliminaries

In [1]:
# Imports
import sys
import json
import pandas as pd
sys.path.append('../0-base-functions/')
import wordstats
from sklearn.feature_extraction.text import CountVectorizer
import random
import numpy as np
import scipy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from io import StringIO
from numpy import nan
from tqdm import tqdm
import rbo

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rlai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Load the GENIA Corpus Data

In particular, we load the
- preprocessed GENIA corpus documents,
- and gold standard biological terms (i.e., lexical units) and their associated semantic classes (i.e., sems).

In [2]:
# Load the preprocessed GENIA corpus documents
genia_corpus_path = '../1-preprocessing/GENIAcorpus3.02-preprocessed.json'

with open(genia_corpus_path, "r") as j:
  genia_corpus = json.loads(j.read())

# Load gold standard terms 
genia_keywords_path = '../1-preprocessing/GENIAcorpus3.02-keywords.tsv'

with open(genia_keywords_path, "r") as c:
  genia_lexical_units_and_sems = pd.read_csv(c, sep='\t')

genia_lexical_units = genia_lexical_units_and_sems.lex.to_numpy()

## Prepare the GENIA Corpus Data for Analysis

Prepare the corpus vocabulary.

In [3]:
# Compile the GENIA corpus vocabulary
pre_vocab = []
for i in range(len(genia_corpus)):
  pre_vocab.append(genia_corpus[i].split())

vocab = []
for i in range(len(pre_vocab)):
  for j in range(len(pre_vocab[i])):
    vocab.append(pre_vocab[i][j])

vocab = list(set(vocab))
vocab.sort()

Convert GENIA documents into term-in-document matrix of token counts.

In [4]:
# Helper function to ensure that CountVectorizer doesn't ignore any terms
def analyzer_custom(doc):
  return doc.split()

# Generate term-in-document counts
counter = CountVectorizer(lowercase=False, vocabulary=vocab, analyzer=analyzer_custom)
collection = counter.transform(genia_corpus)

## Evaluate Term Dispersion Scores for Selected Measures

Calculate bag-of-words model word statistics and related quantities.

In [5]:
# Calculate word statistics and related quantities
m = len(counter.get_feature_names_out()) # vocab size
d = collection.shape[0] # collection size
N_i = wordstats.get_Ni(collection)
N_j = wordstats.get_Nj(collection)
N = wordstats.get_N(N_j)
B_ij = wordstats.get_Bij(collection)
B_i = wordstats.get_Bi(B_ij)
B_j = wordstats.get_Bj(B_ij)
DF = wordstats.get_DF(B_i, d)
CF = wordstats.get_CF(N_i)
nij_by_nj = wordstats.get_nij_by_nj(collection, N_j)
thetas = np.array(range(1, max(N_i.A[0]) + 1))/N
opt_thetas = wordstats.get_opt_thetas(N, m, d, N_i, N_j, B_i, thetas)

Evaluate word dispersion scores.

In [6]:
# Calculate word dispersion scores according the various measures used in this study
IDF = wordstats.get_IDF(DF)
ICF = wordstats.get_ICF(CF)
Chisq = wordstats.get_Chisq(collection)
CG = wordstats.get_CG(N_i, B_i)
ICB = wordstats.get_ICB(nij_by_nj, B_i)
DoP = wordstats.get_DoP(collection, N_i, N_j, N)
RICF = wordstats.get_RICF(opt_thetas, N, ICF)

C:\Users\Rlai\Documents\work\bursty-term-measure\genia\3-tables\../0-base-functions\wordstats.py:209: RuntimeWarning: divide by zero encountered in log
  return -np.log(chisq_values)


Arrange term dispersion scores into a data frame.

In [7]:
# Initialize bursty scores data frame
term_scores_df = pd.DataFrame(data=
                         {'term': counter.get_feature_names_out(),
                          'IDF': IDF.A[0],
                          'ICF': ICF.A[0],
                          'Chi-sq': Chisq,
                          'CG': CG.A[0],
                          'ICB': ICB.A[0],
                          'DoP': DoP.A[0],
                          'RICF': RICF.A[0]})

In [8]:
# Print term dispersion scores to console
display(term_scores_df)

,term,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
0,'aged'_lymphocyte_lex,7.600902,12.800954,0.701595,1.0,244.0,-0.000673,-0.000251
1,'converted'_TCEd_motif_lex,7.600902,12.800954,0.701595,1.0,195.0,-0.000538,-0.000251
2,'latency_I'_lex,7.600902,12.800954,0.701595,1.0,206.0,-0.000568,-0.000251
3,'latency_II'_lex,7.600902,12.800954,0.701595,1.0,206.0,-0.000568,-0.000251
4,'master_regulator_lex,7.600902,12.800954,0.701595,1.0,63.0,-0.000174,-0.000251
...,...,...,...,...,...,...,...,...
40799,zymogen_plasma_factor_X_lex,7.600902,12.800954,0.701595,1.0,355.0,-0.000979,-0.000251
40800,zymogen_plasma_factors_VII_lex,7.600902,12.800954,0.701595,1.0,355.0,-0.000979,-0.000251
40801,zymography_lex,7.600902,12.800954,0.701595,1.0,191.0,-0.000527,-0.000251
40802,zymosan-treated_cell_lex,7.600902,12.800954,0.701595,1.0,225.0,-0.000621,-0.000251


In [9]:
# Write to TSV
term_scores_df.to_csv('term-dispersion-scores.tsv', sep='\t')

## Compile GENIA Corpus Summary Statistics

This is the result of Table 3 from the manuscript.

In [10]:
# Collect lexical units belonging to a given high-level semantic class
def get_high_level_semantic_class_words(high_level_class_lst):
  words = []
  for k, v in lex_sem_dct.items():
    if v in high_level_class_lst:
      words.append(k)
  return words

In [11]:
# Hardcode the semantic classes according to their high-level designations.
amino_acid_sems = ['G#amino_acid_monomer', 'G#peptide', 'G#protein_N/A',
              'G#protein_complex', 'G#protein_domain_or_region',
              'G#protein_family_or_group', 'G#protein_molecule',
              'G#protein_substructure', 'G#protein_subunit',
              'G#other_organic_compound', 'G#organic', 'G#inorganic', 'G#atom',
              'G#carbohydrate', 'G#lipid']
nucleotide_sems = ['G#nucleotide', 'G#polynucleotide', 'G#DNA_N/A',
        'G#DNA_domain_or_region', 'G#DNA_family_or_group', 'G#DNA_molecule',
        'G#DNA_substructure', 'G#RNA_N/A', 'G#RNA_domain_or_region',
        'G#RNA_family_or_group', 'G#RNA_molecule', 'G#RNA_substructure']
multi_cell_sems = ['G#virus', 'G#mono_cell', 'G#multi_cell', 'G#body_part', 'G#tissue']
cell_sems = ['G#cell_type', 'G#cell_component', 'G#cell_line', 'G#other_artificial_source']
other_sems = ['G#other_name']
high_level_semantic_class_names = ['amino_acid', 'nucleotide', 'multi_cell', 'cell', 'other']
high_level_semantic_class_lex_units = [genia_lexical_units, amino_acid_sems, nucleotide_sems, multi_cell_sems, cell_sems, other_sems]

In [12]:
# Create dictionary of lexical units and their associated semantic classes
sem = np.array(genia_lexical_units_and_sems['sem'])
lex = np.array(genia_lexical_units_and_sems['lex'])
lex_sem_dct = dict(zip(lex, sem))

# Count number of semantic classes in each high-level class
sub_class = [len(amino_acid_sems), len(nucleotide_sems), len(multi_cell_sems), len(cell_sems), len(other_sems)]

# Count number of distinct lexical units in each high-level semantic class
amino_acid = get_high_level_semantic_class_words(amino_acid_sems)
nucleotide = get_high_level_semantic_class_words(nucleotide_sems)
multi_cell = get_high_level_semantic_class_words(multi_cell_sems)
cell = get_high_level_semantic_class_words(cell_sems)
other = get_high_level_semantic_class_words(other_sems)

# Count number of lexical units used as annotations for each high-level semantic class
high_level_class_words = [genia_lexical_units, amino_acid, nucleotide, multi_cell, cell, other]
high_level_class_words_counter = [0, 0, 0, 0, 0, 0]
for i in range(len(high_level_class_words)):
  for j in range(len(vocab)):
    if vocab[j] in high_level_class_words[i]:
      high_level_class_words_counter[i] += N_i.A[0][j]

In [13]:
# Create dictionary of lexical units and their associated semantic classes
sem = np.array(genia_lexical_units_and_sems['sem'])
lex = np.array(genia_lexical_units_and_sems['lex'])
lex_sem_dct = dict(zip(lex, sem))

# Count number of semantic subclasses in each high-level class
subclass_counts = [len(amino_acid_sems), len(nucleotide_sems), len(multi_cell_sems), len(cell_sems), len(other_sems)]

# Gather distinct lexical units for each high-level semantic class
amino_acid = get_high_level_semantic_class_words(amino_acid_sems)
nucleotide = get_high_level_semantic_class_words(nucleotide_sems)
multi_cell = get_high_level_semantic_class_words(multi_cell_sems)
cell = get_high_level_semantic_class_words(cell_sems)
other = get_high_level_semantic_class_words(other_sems)

# Count number of lexical units used as annotations for each high-level semantic class
lex_by_semamtic_class_counts = [0, 0, 0, 0, 0, 0]
for i in range(len(high_level_semantic_class_names)):
  for j in range(len(vocab)):
    if vocab[j] in high_level_semantic_class_names[i]:
      lex_by_semamtic_class_counts[i] += N_i.A[0][j]

In [14]:
# Define the data
high_level_semantic_classes = ['all', 'amino_acid', 'nucleotide', 'multi_cell', 'cell', 'other']
annotations_count_zip = zip(high_level_semantic_classes, high_level_class_words_counter)
annotations_dict = dict(annotations_count_zip)
lex_unit_counts = [len(amino_acid), len(nucleotide), len(multi_cell), len(cell), len(other)]
subclass_counts = [len(amino_acid_sems), len(nucleotide_sems), len(multi_cell_sems), len(cell_sems), len(other_sems)]
annotations = [
    annotations_dict['amino_acid'],
    annotations_dict['nucleotide'],
    annotations_dict['multi_cell'],
    annotations_dict['cell'],
    annotations_dict['other']]

# Create a data frame
genia_summary_stats_df = pd.DataFrame({
    'Semantic class': ['amino_acid', 'nucleotide', 'multi_cell', 'cell', 'other'],
    'Sub-class': subclass_counts,
    'Unique terms': lex_unit_counts,
    "Annotations": annotations
})

In [15]:
# Print GENIA summary statistics
display(genia_summary_stats_df)

,Semantic class,Sub-class,Unique terms,Annotations
0,amino_acid,15,10155,42478
1,nucleotide,12,5574,11619
2,multi_cell,5,1444,5247
3,cell,4,4051,11626
4,other,1,10560,19999


In [16]:
# Write to CSV
genia_summary_stats_df.to_csv('table-3/table-3.csv', index=False)

## Terminology Extraction Task Experiment

Here we reproduce the result of Table 4 from the manuscript.

In [50]:
def top_k(dct, k):
  keys = dct.keys()
  values = []
  for key in keys:
    values.append(dct[key][:k])
  keys_values_pair = zip(keys, values)
  return dict(keys_values_pair)
    
def resort(term_scores_df):
    sorted_terms = []
    bursty_measure_names = term_scores_df.columns.values.tolist()[1:]

    for measure in bursty_measure_names:
        # Copy the data frame and add a random column
        temp_df = term_scores_df.copy()
        temp_df['random'] = np.random.rand(len(temp_df))
        
        # Sort by the measure and the random column
        sorted_df = temp_df[['term', measure, 'random']].sort_values(by=[measure, 'random'], ascending=[False, True])
        
        # Append the sorted terms to the list
        sorted_terms.append(np.array(sorted_df['term']))
        
        # Drop the random column from the temporary data frame
        temp_df.drop(columns='random', inplace=True)
    
    sorted_terms = np.array(sorted_terms)
    measure_term_pair = zip(bursty_measure_names, sorted_terms)
    sorted_measures = dict(measure_term_pair)
    return sorted_measures
    
def calculate_means(nested_list):
    result = []
    num_outer = len(nested_list)
    num_inner = len(nested_list[0])

    for i in range(num_inner):
        means = {}
        for column in nested_list[0][i].columns:
            values = [nested_list[outer][i][column].values for outer in range(num_outer)]
            mean_values = np.mean(values, axis=0)
            means[column] = mean_values
        result.append(pd.DataFrame(means, index=nested_list[0][i].index))
    return result

# Function to calculate the standard deviation of the corresponding data frames
def calculate_sds(nested_list):
    result = []
    num_outer = len(nested_list)
    num_inner = len(nested_list[0])

    for i in range(num_inner):
        std_devs = {}
        for column in nested_list[0][i].columns:
            values = [nested_list[outer][i][column].values for outer in range(num_outer)]
            std_values = np.std(values, axis=0, ddof=1)
            std_devs[column] = std_values
        result.append(pd.DataFrame(std_devs, index=nested_list[0][i].index))
    return result

def create_p_k(v, sorted_measures):
  measures = sorted_measures.keys()
  counts = [[], [], [], [], [], [], [], [], [], []]
  p_k_dct = dict(zip(measures, counts))  
  for measure in p_k_dct.keys():
      for value in at_values:
              p_k_dct[measure].append(count_words(top_k(sorted_measures, value)[measure], v)/value)
  result = pd.DataFrame(p_k_dct)
  result.index = at_values
  return result

def create_r_k(v, sorted_measures):
  measures = sorted_measures.keys()
  counts = [[], [], [], [], [], [], [], [], [], []]
  r_k_dct = dict(zip(measures, counts))  
  for measure in r_k_dct.keys():
      for value in at_values:
              r_k_dct[measure].append(count_words(top_k(sorted_measures, value)[measure], v)/value)
  result = pd.DataFrame(r_k_dct)
  result.index = at_values
  return result

def create_f1_k(v, sorted_measures):
  measures = sorted_measures.keys()
  counts = [[], [], [], [], [], [], [], [], [], []]
  f1_k_dct = dict(zip(measures, counts))  
  for measure in f1_k_dct.keys():
      for value in at_values:
              f1_k_dct[measure].append(count_words(top_k(sorted_measures, value)[measure], v)/value)
  result = pd.DataFrame(f1_k_dct)
  result.index = at_values
  return result

def create_RBO(v, sorted_measures):
  RICF = sorted_measures["RICF"]
  measures = sorted_measures.keys()
  counts = [[], [], [], [], [], [], [], [], [], []]
  rbo_dct = dict(zip(measures, counts))  
  for measure in rbo_dct.keys():
      for value in at_values:
          S= top_k(sorted_measures, value)[measure]
          T = RICF[0:value]
          rbo_dct[measure].append(rbo.RankingSimilarity(S, T).rbo())
  result = pd.DataFrame(rbo_dct)
  result.index = at_values
  return result



def count_words(lst, imp_words):
  counter = 0
  for x in lst:
    if x in imp_words:
      counter += 1
  return counter

In [51]:
# Initialize a random seed to ensure results can be replicated
np.random.seed(641369)

# These are the P@k scores for the different categories of domain-specific words
at_values = np.array([10, 50, 100, 500, 1000, 5000])
categories = {'all': genia_lexical_units , 'amino_acid': amino_acid, 'nucleotide': nucleotide, 'multi_cell': multi_cell, 'cell': cell, 'other': other}
all_pk_scores = []
sds_dfs = []
all_rbo_scores = []
# R = 100
R = 1 # For testing purposes; change back to 100 for final results
for r in tqdm(range(R)):
 pk_scores = []
 sorted_measures = resort(term_scores_df)
 print(sorted_measures)
 for k, v in categories.items():
     pk = create_p_k(v, sorted_measures)
     rbo_scores= create_RBO(v, sorted_measures)
     pk_scores.append(pk)
 all_pk_scores.append(pk_scores)
 all_rbo_scores.append(rbo_scores)  
    

  0%|                                                                                    | 0/1 [00:00<?, ?it/s]

{'IDF': array(['recessive', 'helix', 'P3A2_lex', ..., 'in', 'of', 'the'],
      dtype=object), 'ICF': array(['CLE0_element_lex', 'RAP30_lex',
       '12-O-tetradecanoylphorbol-13-acetate_(TPA)_response_element_lex',
       ..., 'in', 'of', 'the'], dtype=object), 'Chi-sq': array(['P_sequence_lex', 'K_protein_lex', 'UAS2_lex', ..., 'indicate',
       'here', 'suggest'], dtype=object), 'CG': array(['Bcl-6_lex', 'SMX_lex', 'v-erbA_lex', ...,
       'signaling_molecules_lex', 'class_I_triggering_lex', 'concurrent'],
      dtype=object), 'ICB': array(['Bcl-6_lex', 'TCRzeta_lex', 'ML-9_lex', ..., 'guardian',
       'interleukin-15_lex', 'thymus-derived_T-cell_homeostasis_lex'],
      dtype=object), 'DoP': array(['guardian', 'distinct_function_lex', 'interleukin-15_lex', ...,
       'in', 'the', 'of'], dtype=object), 'RICF': array(['Bcl-6_lex', 'v-erbA_lex', 'SMX_lex', ..., 'results', 'here',
       'suggest'], dtype=object)}


100%|███████████████████████████████████████████████████████████████████████████| 1/1 [02:43<00:00, 163.23s/it]


In [52]:
sorted_measures
_value = np.array([10, 50, 100, 500, 1000, 5000])
RICF = sorted_measures["RICF"]
for i in sorted_measures.key():
        

SyntaxError: incomplete input (34811852.py, line 5)

In [53]:
# Display the resulting data frames
print("Mean P@k scores:")
with pd.option_context('display.precision', 4):
    display(calculate_means(all_pk_scores)[0])
    display(calculate_means(all_pk_scores)[1])
    display(calculate_means(all_pk_scores)[2])
    display(calculate_means(all_pk_scores)[3])
    display(calculate_means(all_pk_scores)[4])
    display(calculate_means(all_pk_scores)[5])

Mean P@k scores:


,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
10,0.7000,1.0000,0.900,1.0000,1.000,0.9000,1.0000
50,0.9000,0.8800,0.940,0.9600,1.000,0.7800,1.0000
100,0.9000,0.8600,0.940,0.9800,0.980,0.7800,1.0000
500,0.8760,0.8500,0.940,0.9840,0.974,0.8260,0.9920
1000,0.8700,0.8450,0.954,0.9790,0.965,0.8530,0.9850
5000,0.8614,0.8484,0.916,0.9274,0.897,0.8754,0.9316


,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
10,0.3000,0.4000,0.6000,1.000,0.8000,0.100,1.0000
50,0.3000,0.3000,0.5000,0.740,0.7000,0.220,0.7800
100,0.2600,0.3000,0.5100,0.820,0.7400,0.140,0.8300
500,0.2380,0.2660,0.5700,0.686,0.6220,0.220,0.6860
1000,0.2750,0.2540,0.5560,0.640,0.5900,0.253,0.6390
5000,0.2744,0.2494,0.4296,0.428,0.4108,0.280,0.4292


,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
10,0.0000,0.3000,0.2000,0.0000,0.1000,0.0000,0.0000
50,0.2000,0.1400,0.0800,0.1000,0.1200,0.1200,0.1000
100,0.1600,0.1200,0.1000,0.1000,0.1000,0.1400,0.1100
500,0.1780,0.1520,0.1280,0.1440,0.1460,0.1360,0.1460
1000,0.1560,0.1480,0.1430,0.1290,0.1390,0.1390,0.1330
5000,0.1556,0.1512,0.1526,0.1536,0.1482,0.1528,0.1556


,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
10,0.200,0.1000,0.0000,0.000,0.0000,0.0000,0.0000
50,0.040,0.0600,0.0400,0.040,0.0400,0.0800,0.0400
100,0.020,0.0400,0.0400,0.020,0.0200,0.0900,0.0200
500,0.022,0.0400,0.0400,0.034,0.0320,0.0560,0.0360
1000,0.027,0.0330,0.0480,0.037,0.0420,0.0530,0.0370
5000,0.037,0.0376,0.0436,0.044,0.0412,0.0468,0.0442


,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
10,0.1000,0.0000,0.1000,0.0000,0.0000,0.1000,0.000
50,0.1000,0.0600,0.1600,0.0200,0.0000,0.1600,0.020
100,0.1200,0.0900,0.1400,0.0100,0.0200,0.1200,0.010
500,0.1420,0.1040,0.0900,0.0400,0.0600,0.0900,0.038
1000,0.1300,0.1100,0.0890,0.0610,0.0700,0.0920,0.062
5000,0.1092,0.1078,0.0952,0.1004,0.0986,0.0948,0.101


,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
10,0.1000,0.2000,0.000,0.0000,0.1000,0.700,0.0000
50,0.2600,0.3200,0.160,0.0600,0.1400,0.200,0.0600
100,0.3400,0.3100,0.150,0.0300,0.1000,0.290,0.0300
500,0.2960,0.2880,0.112,0.0800,0.1140,0.324,0.0860
1000,0.2820,0.3000,0.118,0.1120,0.1240,0.316,0.1140
5000,0.2852,0.3024,0.195,0.2014,0.1982,0.301,0.2016


In [55]:
print(all_rbo_scores)

# with pd.option_context('display.precision', 4):
#     display(calculate_means(all_rbo_scores)[0])
#     display(calculate_means(all_rbo_scores)[1])
#     display(calculate_means(all_rbo_scores)[2])
#     display(calculate_means(all_rbo_scores)[3])
#     display(calculate_means(all_rbo_scores)[4])
#     display(calculate_means(all_rbo_scores)[5])

[           IDF  ICF    Chi-sq        CG       ICB       DoP  RICF
10    0.000000  0.0  0.000000  0.877103  0.602421  0.000000   1.0
50    0.000000  0.0  0.000000  0.894210  0.688419  0.000000   1.0
100   0.000000  0.0  0.001565  0.903774  0.699389  0.000000   1.0
500   0.006248  0.0  0.101921  0.930891  0.725895  0.000543   1.0
1000  0.014245  0.0  0.205158  0.934544  0.764003  0.002995   1.0
5000  0.061572  0.0  0.656543  0.905749  0.795456  0.042883   1.0]


In [21]:
print("P@k scores standard deviations:")
with pd.option_context('display.precision', 4):
    display(calculate_sds(all_pk_scores)[0])
    display(calculate_sds(all_pk_scores)[1])
    display(calculate_sds(all_pk_scores)[2])
    display(calculate_sds(all_pk_scores)[3])
    display(calculate_sds(all_pk_scores)[4])
    display(calculate_sds(all_pk_scores)[5])

P@k scores standard deviations:


,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
10,0.0577,0.1732,0.0000,0.0000,0.0000e+00,0.0000e+00,0.0000
50,0.0611,0.0306,0.0306,0.0000,0.0000e+00,1.1547e-02,0.0000
100,0.0473,0.0361,0.0252,0.0000,0.0000e+00,1.3597e-16,0.0000
500,0.0186,0.0053,0.0050,0.0012,1.3597e-16,1.1547e-03,0.0000
1000,0.0095,0.0075,0.0035,0.0023,1.0000e-03,5.7735e-04,0.0021
5000,0.0030,0.0009,0.0011,0.0003,2.0000e-04,1.1547e-04,0.0002


,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
10,0.1528,0.1155,0.1000,0.0000,1.3597e-16,0.0577,0.0000
50,0.0872,0.0306,0.0808,0.0115,1.3597e-16,0.0115,0.0115
100,0.0577,0.0115,0.0529,0.0000,5.7735e-03,0.0058,0.0000
500,0.0231,0.0110,0.0361,0.0023,0.0000e+00,0.0000,0.0053
1000,0.0124,0.0072,0.0190,0.0006,5.7735e-04,0.0006,0.0051
5000,0.0051,0.0059,0.0008,0.0010,2.3094e-04,0.0000,0.0004


,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
10,0.1528,0.1732,0.1155,0.0000e+00,1.6997e-17,0.0000,0.0000e+00
50,0.0872,0.0306,0.0611,1.6997e-17,0.0000e+00,0.0000,1.6997e-17
100,0.0529,0.0231,0.0755,1.6997e-17,1.6997e-17,0.0058,0.0000e+00
500,0.0227,0.0020,0.0156,1.1547e-03,0.0000e+00,0.0012,1.1547e-03
1000,0.0118,0.0090,0.0069,4.3589e-03,5.7735e-04,0.0010,1.0000e-03
5000,0.0029,0.0076,0.0005,1.1136e-03,1.1547e-04,0.0002,2.3094e-04


,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
10,0.1000,0.0577,0.0000,0.0000,0.0000,0.0000e+00,0.0000
50,0.0200,0.0200,0.0200,0.0000,0.0000,0.0000e+00,0.0000
100,0.0173,0.0100,0.0058,0.0000,0.0000,1.6997e-17,0.0000
500,0.0101,0.0050,0.0031,0.0012,0.0000,0.0000e+00,0.0023
1000,0.0085,0.0064,0.0057,0.0010,0.0000,0.0000e+00,0.0012
5000,0.0009,0.0040,0.0006,0.0004,0.0001,0.0000e+00,0.0002


,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
10,0.0577,0.1155,1.6997e-17,0.0000,0.0000e+00,5.7735e-02,0.0000
50,0.0306,0.0872,5.0332e-02,0.0115,0.0000e+00,1.1547e-02,0.0115
100,0.0252,0.0529,2.0817e-02,0.0000,0.0000e+00,0.0000e+00,0.0000
500,0.0223,0.0064,1.2055e-02,0.0012,0.0000e+00,1.6997e-17,0.0020
1000,0.0075,0.0050,7.8102e-03,0.0025,5.7735e-04,1.6997e-17,0.0036
5000,0.0052,0.0018,2.0000e-04,0.0003,1.6997e-17,1.1547e-04,0.0003


,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
10,0.1732,0.1732,0.0577,0.0000,1.6997e-17,5.7735e-02,0.0000
50,0.0400,0.0721,0.0600,0.0000,0.0000e+00,3.3993e-17,0.0000
100,0.0321,0.0400,0.0100,0.0000,5.7735e-03,0.0000e+00,0.0000
500,0.0081,0.0081,0.0200,0.0012,0.0000e+00,1.1547e-03,0.0020
1000,0.0108,0.0083,0.0153,0.0015,1.1547e-03,0.0000e+00,0.0021
5000,0.0062,0.0093,0.0015,0.0007,2.3094e-04,3.0551e-04,0.0002


In [22]:
# Calculate mean P@k scores and write to CSV
pd.DataFrame(calculate_means(all_pk_scores)[0]).to_csv('table-4/table-4-all-means.csv', index=False)
pd.DataFrame(calculate_means(all_pk_scores)[1]).to_csv('table-4/table-4-amino_acid-means.csv', index=False)
pd.DataFrame(calculate_means(all_pk_scores)[2]).to_csv('table-4/table-4-nucleotide-means.csv', index=False)
pd.DataFrame(calculate_means(all_pk_scores)[3]).to_csv('table-4/table-4-multicell-means.csv', index=False)
pd.DataFrame(calculate_means(all_pk_scores)[4]).to_csv('table-4/table-4-cell-means.csv', index=False)
pd.DataFrame(calculate_means(all_pk_scores)[5]).to_csv('table-4/table-4-other-means.csv', index=False)

# Calculate standard deviations for P@k scores and write to CSV
pd.DataFrame(calculate_sds(all_pk_scores)[0]).to_csv('table-4/table-4-all-sds.csv', index=False)
pd.DataFrame(calculate_sds(all_pk_scores)[1]).to_csv('table-4/table-4-amino_acid-sds.csv', index=False)
pd.DataFrame(calculate_sds(all_pk_scores)[2]).to_csv('table-4/table-4-nucleotide-sds.csv', index=False)
pd.DataFrame(calculate_sds(all_pk_scores)[3]).to_csv('table-4/table-4-multicell-sds.csv', index=False)
pd.DataFrame(calculate_sds(all_pk_scores)[4]).to_csv('table-4/table-4-cell-sds.csv', index=False)
pd.DataFrame(calculate_sds(all_pk_scores)[5]).to_csv('table-4/table-4-other-sds.csv', index=False)

## Top 10 Ranked Terms Example

Here we reproduce the result of Table 5 from the manuscript.

In [23]:
# Initialize a random seed to ensure results can be replicated
np.random.seed(641369)

In [24]:
ranked_terms_df = resort(term_scores_df)
top_10_ranked_terms_df = pd.DataFrame(top_k(ranked_terms_df, 10))
display(top_10_ranked_terms_df)

,IDF,ICF,Chi-sq,CG,ICB,DoP,RICF
0,recessive,CLE0_element_lex,P_sequence_lex,Bcl-6_lex,Bcl-6_lex,guardian,Bcl-6_lex
1,helix,RAP30_lex,K_protein_lex,SMX_lex,TCRzeta_lex,distinct_function_lex,v-erbA_lex
2,P3A2_lex,12-O-tetradecanoylphorbol-13-acetate_(TPA)_res...,UAS2_lex,v-erbA_lex,ML-9_lex,interleukin-15_lex,SMX_lex
3,p0005,AIDS_IBLP_tumor_lex,Bik_lex,ML-9_lex,AITL_lex,thymus-derived_T-cell_homeostasis_lex,SHP1_lex
4,obese_patient_lex,beta-like_globin_cluster_lex,sesquiterpene_lactone_lex,SHP1_lex,SHP1_lex,extrathymic_development_lex,ML-9_lex
5,human_activated_monocyte_lex,stromal-derived_cytokine_interleukin-7_lex,TS_lex,beta-casein_lex,beta-casein_lex,B-lymphocyte_differentiation_control_lex,beta-casein_lex
6,putative_chicken_Shc_homologue_lex,inductive_stimulation_lex,MTBE_lex,EBNA-2_lex,A-myb_lex,B-lymphocyte_differentiation_lex,I_kappaB_lex
7,DNA_ploidy_lex,Myc_lex,Tax_lex,DM_lex,I_kappaB_lex,plasma_cell_pathway_lex,DM_lex
8,mature_organ_lex,N-terminal_c-Jun_kinase_lex,variants,TCRzeta_lex,SMX_lex,B-cell_commitment_lex,TCRzeta_lex
9,small_GTP-binding_protein_Rho_lex,PKR_expression_lex,glial_cell_lex,I_kappaB_lex,Rap1_protein_lex,memory_B_cells_lex,p95vav_lex


In [25]:
# Write to CSV
top_10_ranked_terms_df.to_csv('table-5/table-5.csv', index=False)

## Stopwords Exploratory Analysis

Here we reproduce the result of Table 6 from the manuscript.

In [26]:
import pandas as pd
from nltk.corpus import stopwords

# Ensure you have the stopwords downloaded
import nltk
nltk.download('stopwords')

def getrank(sorted_measures):
    unique_terms = set()
    for terms in sorted_measures.values():
        unique_terms.update(terms)
    unique_terms = sorted(unique_terms)
    
    # Create a data frame to hold the rankings
    ranking_df = pd.DataFrame(index=unique_terms, columns=sorted_measures.keys())
    
    # Fill the data frame with rankings
    for measure, terms in sorted_measures.items():
        for rank, term in enumerate(terms):
            ranking_df.at[term, measure] = rank + 1  # Rank starts from 1
    
    # Replace NaN with a large number to indicate unranked terms
    ranking_df = ranking_df.fillna(len(unique_terms) + 1)
    #csv_file_path = 'ranking_table.csv'
    #ranking_df.to_csv(csv_file_path)
    return ranking_df

# Function to filter stopwords from the ranking data frame
def filter_stopwords(ranking_df):
    stopwords_list = set(stopwords.words('english'))
    
    # Filter the data frame to include only stopwords
    stopwords_rank = ranking_df[ranking_df.index.isin(stopwords_list)]
    
    # Save the stopwords ranking data frame to a CSV file
    #csv_file_path = 'stopwords_ranking_table.csv'
    #stopwords_rank.to_csv(csv_file_path)
    
    return stopwords_rank

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


In [27]:
# Initialize a random seed to ensure results can be replicated
np.random.seed(641369)

# Generate term dispersion ranks for R different versions of the data
R = 100
all_quantiles_df = []
for r in tqdm(range(R)):
    sorted_measures = resort(term_scores_df)
    rank = getrank(sorted_measures)
    stopwords_ranks_df = filter_stopwords(rank)
    bursty_measure_names = stopwords_ranks_df.head(0)
    quantiles = []
    for bursty_measure_name in bursty_measure_names:
        quantiles.append(stopwords_ranks_df[bursty_measure_name].quantile([0, 0.25, 0.5, 0.75, 1]))
    quantiles_df = pd.DataFrame(quantiles)
    all_quantiles_df.append(quantiles_df)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:03<00:00,  1.24s/it]


In [28]:
# Extract the column and index names from the first quantiles data frame
columns = all_quantiles_df[0].columns
index = all_quantiles_df[0].index

# Initialize empty data frames to store the mean and standard deviation values
mean_df = pd.DataFrame(index=index, columns=columns)
std_df = pd.DataFrame(index=index, columns=columns)

# Compute the mean and standard deviation of corresponding elements across all matrices
for col in columns:
    for idx in index:
        values = [matrix.at[idx, col] for matrix in all_quantiles_df]
        mean_df.at[idx, col] = np.mean(values)
        std_df.at[idx, col] = np.std(values)

In [29]:
# Display the resulting data frames
print("Mean values:")
with pd.option_context('display.precision', 4):
    display(mean_df)
print("\nStandard deviations:")
with pd.option_context('display.precision', 4):
    display(std_df)

Mean values:


,0.00,0.25,0.50,0.75,1.00
IDF,4184.62,39839.62,40595.55,40765.0,40804.0
ICF,4066.6,39749.07,40564.31,40760.0,40804.0
Chi-sq,947.7,7402.0,8077.0,8736.0,40803.0
CG,19.0,6318.0,7748.58,8619.0,39282.65
ICB,70.0,6042.0,12018.0,17067.0,36577.55
DoP,5059.86,39858.0,40602.0,40765.0,40804.0
RICF,2427.0,7804.0,8287.0,8844.0,40803.0



Standard deviations:


,0.00,0.25,0.50,0.75,1.00
IDF,3330.9626,8.6218,0.4975,0.0,0.0
ICF,3688.3649,5.8826,0.4625,0.0,0.0
Chi-sq,566.6572,0.0,0.0,0.0,0.0
CG,0.0,0.0,0.4936,0.0,1455.7712
ICB,0.0,0.0,0.0,0.0,52.6453
DoP,7.985,0.0,0.0,0.0,0.0
RICF,0.0,0.0,0.0,0.0,0.0


In [30]:
# Write to CSV
mean_df.to_csv('table-6/table-6-means.csv')
std_df.to_csv('table-6/table-6-sds.csv')